In [80]:
import cv2
import numpy as np
import csv
import math
import time
from scipy.spatial import distance_matrix

In [62]:
map_index = 1
file_name = f'perumdos-{map_index}.png'

map_image = cv2.imread(file_name, cv2.IMREAD_COLOR)
map_image = cv2.resize(map_image, (1920, 1080))
img = map_image.copy()

coordinates = []

with open(f'coordinates-{map_index}.csv', mode='r') as file:
  reader = csv.reader(file)
  next(reader)
  for row in reader:
    x, y = int(row[1]), int(row[2])
    coordinates.append((x, y))
    cv2.circle(img, (x, y), 5, (0, 0, 255), -1)

print(coordinates)

[]


In [64]:
def print_coordinates(event, x, y, flags, param):
	if event == cv2.EVENT_LBUTTONDOWN:
		coordinates.append((x, y))
		# print(f"[{x}, {y}],")
		cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
		cv2.imshow("Image", img)

cv2.imshow("Image", img)
cv2.setMouseCallback("Image", print_coordinates)

while True:
	key = cv2.waitKey(1) & 0xFF

	if key == ord('d'):
		if len(coordinates) > 0:
			coordinates.pop()
			img = map_image.copy()
			for coord in coordinates:
				cv2.circle(img, coord, 5, (0, 0, 255), -1)
			cv2.imshow("Image", img)

	elif key == 27:
		break

cv2.destroyAllWindows()

with open(f'coordinates-{map_index}.csv', mode='w', newline='') as file:
	writer = csv.writer(file)
	writer.writerow(['No', 'X', 'Y'])
	for i, coord in enumerate(coordinates):
		writer.writerow([i, coord[0], coord[1]])

print(coordinates)

[(709, 84), (765, 85), (817, 87), (845, 88), (903, 89), (931, 90), (968, 89), (1014, 98), (1049, 69), (1075, 90), (1109, 88), (1142, 88), (1189, 87), (1230, 92), (737, 151), (733, 190), (733, 236), (736, 270), (738, 305), (737, 332), (736, 361), (736, 398), (739, 442), (738, 471), (735, 504), (739, 527), (735, 566), (769, 566), (788, 579), (784, 540), (778, 504), (782, 468), (782, 447), (784, 396), (786, 358), (782, 331), (781, 304), (782, 268), (783, 242), (783, 212), (781, 184), (780, 145), (841, 153), (843, 186), (842, 212), (847, 241), (852, 278), (842, 301), (848, 328), (839, 362), (840, 389), (836, 424), (843, 442), (842, 479), (841, 500), (840, 522), (841, 560), (837, 579), (892, 562), (890, 536), (890, 505), (890, 475), (890, 446), (890, 423), (892, 384), (895, 359), (891, 331), (889, 303), (890, 264), (890, 211), (889, 182), (886, 153), (957, 146), (948, 200), (947, 244), (946, 273), (938, 310), (938, 327), (951, 393), (949, 424), (949, 448), (958, 484), (993, 529), (997, 495)

In [65]:
# Fungsi untuk menghitung jarak Euclidean antara dua titik (x1, y1) dan (x2, y2)
def euclidean_distance(p1, p2):
  return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

In [66]:
point1 = [
  (657, 156),
  (829, 177),
  (294, 218)
]

point2 = [
  (709, 147),
  (834, 217),
  (425, 219)
]

real_distance = [
  28.78,
  17.9,
  45.3
]

scale = real_distance[map_index-1] / euclidean_distance(point1[map_index-1], point2[map_index-1])
print(scale)

0.3457914519429337


In [77]:
# Find distance between every pair of points
distances = distance_matrix(coordinates, coordinates)
n = len(coordinates)

In [91]:
# Prim's Algorithm

start_time = time.time() # Start Timer

mst_edges = []
visited = [False] * n
edge_costs = [float('inf')] * n
parent = [-1] * n

edge_costs[0] = 0

for _ in range(n):
  min_cost = float('inf')
  u = -1
  for v in range(n):
    if not visited[v] and edge_costs[v] < min_cost:
      min_cost = edge_costs[v]
      u = v

  visited[u] = True

  if parent[u] != -1:
    mst_edges.append((parent[u], u))

  for v in range(n):
    if not visited[v] and distances[u][v] < edge_costs[v]:
      edge_costs[v] = distances[u][v]
      parent[v] = u

end_time = time.time() # End Timer
elapsed_time = end_time - start_time

img_result_prim = cv2.imread(file_name, cv2.IMREAD_COLOR)
img_result_prim = cv2.resize(img_result_prim, (1920, 1080))

print("Edges in MST:")
total_weight = 0
for edge in mst_edges:
  u, v = edge
  weight = distances[u][v]
  total_weight += weight
  print(f"{u} -> {v} with distance {weight*scale:.2f}m")
print(f"Total distance of MST: {total_weight*scale:.2f}m")
print(f"Elapsed Time: {elapsed_time*100:.3f}ms")

for index, coord in enumerate(coordinates):
  cv2.circle(img_result_prim, tuple(coord), 5, (0, 0, 255), -1)
  cv2.putText(img_result_prim, str(index + 1), tuple(coord), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

for edge in mst_edges:
  pt1 = tuple(coordinates[edge[0]])
  pt2 = tuple(coordinates[edge[1]])
  cv2.line(img_result_prim, pt1, pt2, (255, 0, 0), 2)

cv2.imshow("MST Prim Result", img_result_prim)
cv2.waitKey(0)
cv2.destroyAllWindows()

Edges in MST:
0 -> 1 with distance 19.37m
1 -> 2 with distance 17.99m
2 -> 3 with distance 9.69m
3 -> 4 with distance 20.06m
4 -> 5 with distance 9.69m
5 -> 6 with distance 12.80m
6 -> 7 with distance 16.21m
7 -> 8 with distance 15.72m
8 -> 9 with distance 11.56m
9 -> 10 with distance 11.78m
10 -> 11 with distance 11.41m
11 -> 12 with distance 16.26m
12 -> 13 with distance 14.28m
10 -> 106 with distance 16.96m
106 -> 105 with distance 8.22m
105 -> 104 with distance 9.78m
104 -> 103 with distance 10.03m
103 -> 102 with distance 8.99m
102 -> 101 with distance 10.32m
101 -> 100 with distance 9.02m
100 -> 99 with distance 14.01m
106 -> 93 with distance 14.26m
93 -> 94 with distance 11.07m
94 -> 95 with distance 10.43m
95 -> 96 with distance 10.38m
100 -> 97 with distance 14.28m
97 -> 98 with distance 12.47m
95 -> 90 with distance 19.02m
90 -> 91 with distance 11.16m
91 -> 92 with distance 10.08m
92 -> 72 with distance 12.34m
90 -> 89 with distance 14.87m
91 -> 73 with distance 15.48m
73 ->

In [94]:
# Kruskal's Algorithm

start_time = time.time() # Start Timer

edges = []
for i, coord1 in enumerate(coordinates):
  for j, coord2 in enumerate(coordinates):
    if i < j:
      dist = distances[i][j]
      edges.append((dist, i, j))

edges.sort()

parent = list(range(len(coordinates)))
rank = [0] * len(coordinates)

def find(x):
  if parent[x] != x:
    parent[x] = find(parent[x])
  return parent[x]

def union(x, y):
  rootX = find(x)
  rootY = find(y)

  if rootX != rootY:
    if rank[rootX] > rank[rootY]:
      parent[rootY] = rootX
    elif rank[rootX] < rank[rootY]:
      parent[rootX] = rootY
    else:
      parent[rootY] = rootX
      rank[rootX] += 1

mst_edges = []
total_weight = 0

for dist, u, v in edges:
  if find(u) != find(v):
    union(u, v)
    mst_edges.append((u, v, dist))
    total_weight += dist

end_time = time.time() # End Timer
elapsed_time = end_time - start_time

print("Edges in MST:")
for edge in mst_edges:
  print(f"{edge[0]} -> {edge[1]} with distance {edge[2] * scale:.2f}")
print(f"Total distance of MST: {total_weight * scale:.2f}")
print(f"Elapsed Time: {elapsed_time*100:.3f}ms")

img_result_kruskal = cv2.imread(file_name, cv2.IMREAD_COLOR)
img_result_kruskal = cv2.resize(img_result_kruskal, (1920, 1080))

for index, node in enumerate(coordinates):
  cv2.circle(img_result_kruskal, node, 5, (0, 0, 255), -1)
  cv2.putText(img_result_kruskal, str(index+1), node, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

for edge in mst_edges:
  pt1 = coordinates[edge[0]]
  pt2 = coordinates[edge[1]]
  cv2.line(img_result_kruskal, pt1, pt2, (255, 0, 0), 2)

cv2.imshow("MST Kruskal Result", img_result_kruskal)
cv2.waitKey(0)
cv2.destroyAllWindows()

Edges in MST:
76 -> 77 with distance 5.88
51 -> 52 with distance 6.68
56 -> 57 with distance 6.71
125 -> 127 with distance 7.13
31 -> 32 with distance 7.26
53 -> 54 with distance 7.27
110 -> 111 with distance 7.39
54 -> 55 with distance 7.62
62 -> 63 with distance 7.95
27 -> 28 with distance 7.96
24 -> 25 with distance 8.07
105 -> 106 with distance 8.22
79 -> 80 with distance 8.30
127 -> 128 with distance 8.64
150 -> 151 with distance 8.65
131 -> 132 with distance 8.67
46 -> 47 with distance 8.67
64 -> 65 with distance 8.71
102 -> 103 with distance 8.99
37 -> 38 with distance 9.00
43 -> 44 with distance 9.00
58 -> 59 with distance 9.02
100 -> 101 with distance 9.02
130 -> 131 with distance 9.08
124 -> 125 with distance 9.31
140 -> 141 with distance 9.31
18 -> 19 with distance 9.34
35 -> 36 with distance 9.34
49 -> 50 with distance 9.34
87 -> 88 with distance 9.34
34 -> 35 with distance 9.44
139 -> 140 with distance 9.50
47 -> 48 with distance 9.56
154 -> 155 with distance 9.68
2 -> 3 w

In [95]:
# Displaying Comparison

crop_size = [
  (500, 1400),
  (400, 1450),
  (500, 1400)
]

image1 = img_result_prim.copy()
image2 = img_result_kruskal.copy()

height, width, _ = image1.shape

image1 = image1[0:height, crop_size[map_index-1][0]:crop_size[map_index-1][1]]
image2 = image2[0:height, crop_size[map_index-1][0]:crop_size[map_index-1][1]]

cv2.rectangle(image1, (0, 0), (image1.shape[1], 50), (0, 0, 0), -1)
cv2.rectangle(image2, (0, 0), (image2.shape[1], 50), (0, 0, 0), -1)

cv2.putText(image1, "Prim's", (int(image1.shape[1]*0.5) - 20, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(image2, "Kruskal's", (int(image2.shape[1]*0.5) - 50, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

comparison = np.hstack((image1, image2))
cv2.line(comparison, (image1.shape[1], 0), (image1.shape[1], height), (0, 0, 0), 2)

height, width, _ = comparison.shape
new_height = int(height * 0.9)
new_width = int(width * 0.9)
comparison = cv2.resize(comparison, (new_width, new_height))

cv2.imshow("Comparison", comparison)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [76]:
# Save Result
cv2.imwrite(f"result-{map_index}.png", comparison)

True